# Summary
This notebook shows how to apply the proposed algorithms.

In [1]:
import numpy as np

from generator import process_singlescale, process_multiscale
from metrics import count_accuracy_complete
from models import MSCASTLE, SSCASTLE
from numpy.random import seed 

## SS-CASTLE

In [2]:
N=5
T=512
beta = 1. #non-Gaussian

In [3]:
seed(seed=123456789)
Y, C = process_singlescale(N,T,beta)

In [4]:
ssm = SSCASTLE(Y.T, lag=None)
print("Estimated lag: {}; Ground truth: {}".format(ssm.lag, C.shape[0]-1))

Bs, _ = ssm.solver(reg='l1', thresh=.1, interval=(-1.,1.), lmbd=1.e-3, h_tol=1.e-8, verbose=False)

#in the ground truth, parents are on the columns,
#whereas in the estimate on the rows.
count_accuracy_complete(np.expand_dims(C.transpose(0,2,1),0), np.expand_dims(Bs,0))

Estimated lag: 1; Ground truth: 1


{'accuracy': 0.9848484848484849,
 'fdr|(1-precision)': 0.05555555555555558,
 'tpr|recall': 1.0,
 'fpr': 0.030303030303030304,
 'f1': 0.9714285714285714,
 'nnz': 18,
 'true_nnz': 17,
 'shd-missing': 0,
 'shd-extra': 1,
 'shd-reverse': 0,
 'shd': 1,
 'shd/true_nnz': 0.058823529411764705}

## MS-CASTLE

In [5]:
transform='dwt'
wv='db5'

In [6]:
seed(seed=123456789)
Y, Y_mra, eps_mra, C_true = process_multiscale(N, T, beta, transform=transform, wv=wv) 

In [7]:
model = MSCASTLE(Y.T, lag=1, multiscale=True, transform='swt', wavelet='db5', ndetails=3)
Bs = model._solver(reg='l1', thresh=.1, interval=(-1.,1.), lmbd=1.e-2, h_tol=1.e-10, verbose=False)
count_accuracy_complete(C_true, Bs.transpose((0,1,3,2)))

{'accuracy': 0.8449790512231383,
 'fdr|(1-precision)': 0.2909090909090909,
 'tpr|recall': 0.7959183673469388,
 'fpr': 0.10596026490066225,
 'f1': 0.75,
 'nnz': 55,
 'true_nnz': 49,
 'shd-missing': 8,
 'shd-extra': 14,
 'shd-reverse': 2,
 'shd': 24,
 'shd/true_nnz': 0.4897959183673469}